##### <span style="color:red">Name:</span>

Hiranmayi Duvvuri
hduvvuri

# <span style="color:teal;">CIS 211 Project 5:  Blackjack</span>

##### Due 11:00 P.M. Tuesday May 5

##### Reading:  Perkovic Ch 9 (plus a program in Sec 6.5)

This week's project is to write a Python program that plays Blackjack.  The program will have a GUI implemented with Tk.

For this week's project there is just one markdown cell to fill in for documentation.  Use the markdown cell to explain what you did to create your GUI, how you tested your program, and anything that is interesting about your project.

### <span style="color:teal">Setup</span>

Make sure you have the following files in your project directory:
* the Card class from Project 2 (either copy your own Card.py class, or download `Card.pyc` from Canvas)
* the Deck class from Project 3 (your own, or download `Deck.pyc` from Canvas)
* the CardLabel class and cardimages folder, download from Canvas

To test a GUI program from an IPython Notebook execute the following code cell each time you open this notebook or reset the kernel:

In [1]:
%gui tk

from tkinter import *
from tkinter.messagebox import *

If you are going to use IPython, use this statement to import the CardLabel class:

In [2]:
from CardLabel import CardTextLabel as CardLabel
from Card import *
from Deck import *
from random import shuffle

### <span style="color:teal">Programming Project</span>

Write a program that will play a game of Blackjack. 

The top level window in the GUI for the program should have room for two rows of cards, with six cards in each row. Below the cards display three buttons, named “deal”, “hit”, and “stand”.

The Deal button should start a new game. Shuffle a deck of cards, then display two cards for the dealer in the top row, with one card face down, and display two cards for the player, both face up, in the bottom row. The remaining four places in each row should be blank cards.  This is the initial display (using the CardTextLabel class to work around the IPython image bug):

<img src="http://www.cs.uoregon.edu/Classes/15S/cis211/images/blackjack_hand.png"/>

The Hit button should add a card, face up, in the bottom row, and it should update the player’s score using a function named `total` (described below). If the score is over 21 the player loses the game, and your program should display an alert box with a consoling message.

If the user clicks the Stand button, turn over the dealer’s hidden card and compute the total points for the cards in the dealer’s row. Then while the dealer’s total score is less than 17 add cards (displayed face up) to the dealer’s hand and add the points for that card to the dealer’s total. When the dealer’s total is 17 or higher compare the dealer’s total with the player’s total and display a message that shows the result of the game (dealer wins, player wins, or tie game).

### <span style="color:teal">Counting Points</span>


In Blackjack aces can be either 11 points or 1 point.  The function that computes the total number of points in a hand needs to "optimize" the assignment of points for each ace.

An algorithm on page 201 (Sec 6.5) of the textbook explains how to compute the points in a hand.  You need to adapt this so it works with our Card objects.

The idea is to use a loop that computes an initial total, using 11 points for each ace and counting the number of aces in the hand. Then a second loop changes aces to 1 point if necessary: while the number of aces is greater than 0 and the total is greater than 21 subtract 10 from the total and subtract one from the ace counter.

### <span style="color:teal">Define a New Frame Class</span>

For full credit, your GUI should be fully encapsulated inside a new class named BlackjackFrame, derived from Tk's Frame class.

You can write a program that uses global variables as names of widgets and callbacks, but there will be a **10% penalty.**

### <span style="color:teal">Variable Names</span>

<span style="color:red; font-weight:bold;">Important:</span> The automated test program we will use for this project requires you to use the following variable names:
* use `cards` as the name of the deck of cards
* use `player_hand` and `dealer_hand` for the names of the lists of cards for each player
* use `dealcb`, `hitcb`, and `standcb` as the names for the callback functions for the Deal, Hit, and Stand buttons

Note: If you are putting all your code inside the BlackjackFrame class, the list names should all be preceded by an underscore: `_cards`, `_player_hand`, and `_dealer_hand`.

### <span style="color:teal">Your Program</span> 

##### <span style="color:red">Documentation:</span>

Testing was done by using print statements in each function to make sure objects were appended to lists properly, to track move, and making sure the game was reset properly when Deal button is pressed. Each function tested as it was written.

BlackjackFrame contains four functions, `deal` which resets the game, `hit` uses deal from Deck() to add new cards to a players hand, `stand` compares the dealer and players hands after using deal from Deck() to add cards to the dealers hands, and `total` calculates the points in a given hand which is used in both `hit` and `stand` to trigger the appropriate win or lose message boxes based on how many points each hand has.

##### <span style="color:red">Program:</span>

In [16]:
class BlackjackFrame(Frame):
    """
    Subclass of Frame, creates a blackjack game.
    """
    
    _player_hand = []
    _dealer_hand = []
    
    _player_move = 0
    _dealer_move = 0
    
    _player_wins = 0
    _dealer_wins = 0
    
    def __init__(self, parent):
        """
        Buttons and card labels initialized.
        """
        
        Frame.__init__(self, parent)
        self.configure(background = 'white')
        
        self._cards = Deck(BlackjackCard)
        self._cards.shuffle()
        self.pack()
        
        CardLabel.load_images()
        
        # dealers cards
        self._dc1 = CardLabel(self)
        self._dc1.grid(row = 0, column = 0)
        self._dc2 = CardLabel(self)
        self._dc2.grid(row = 0, column = 1)
        self._dc3 = CardLabel(self)
        self._dc3.grid(row = 0, column = 2)
        self._dc4 = CardLabel(self)
        self._dc4.grid(row = 0, column = 3)
        self._dc5 = CardLabel(self)
        self._dc5.grid(row = 0, column = 4)
        self._dc6 = CardLabel(self)
        self._dc6.grid(row = 0, column = 5)
        
        # players cards
        self._pc1 = CardLabel(self)
        self._pc1.grid(row = 1, column = 0)
        self._pc2 = CardLabel(self)
        self._pc2.grid(row = 1, column = 1)
        self._pc3 = CardLabel(self)
        self._pc3.grid(row = 1, column = 2)
        self._pc4 = CardLabel(self)
        self._pc4.grid(row = 1, column = 3)
        self._pc5 = CardLabel(self)
        self._pc5.grid(row = 1, column = 4)
        self._pc6 = CardLabel(self)
        self._pc6.grid(row = 1, column = 5)
        
        self._deal = Button(self, text = 'Deal', command = self.dealcb)
        self._deal.grid(row = 2, column = 0, padx = 10, pady = 10)
        self._hit = Button(self, text = 'Hit', command = self.hitcb)
        self._hit.grid(row = 2, column = 2, padx = 10, pady = 10)
        self._stand = Button(self, text = 'Stand', command = self.standcb)
        self._stand.grid(row = 2, column = 4, padx = 10, pady = 10)

        self.dealcb()
        
    def dealcb(self):
        """
        Resets player and dealer hands with 2 cards each from top of deck and sets up starting layout for each hand.
        """
        
        self._hit.configure(state = ACTIVE)
        self._stand.configure(state = ACTIVE)
        
        # put cards back at end of deck.
        old = self._dealer_hand + self._player_hand
        self._cards.restore(old)

        self._player_hand = self._cards.deal(2)
        self._dealer_hand = self._cards.deal(2)
        self._player_move = 0
        self._dealer_move = 0
        
        # dealers cards setup
        self._dc1.display('back')
        self._dc2.display('front', self._dealer_hand[1]._id)
        self._dc3.display('blank')
        self._dc4.display('blank')
        self._dc5.display('blank')
        self._dc6.display('blank')
        
        # players cards setup
        self._pc1.display('front', self._player_hand[0]._id)
        self._pc2.display('front', self._player_hand[1]._id)
        self._pc3.display('blank')
        self._pc4.display('blank')
        self._pc5.display('blank')
        self._pc6.display('blank')
        
        # dealer and player win count
        Label(self, text = 'Dealer Wins: ').grid(row = 3, column = 2, padx = 10, pady = 10)
        self.dcount = Label(self, text = self._dealer_wins)
        self.dcount.update_idletasks()
        self.dcount.grid(row = 3, column = 3, padx = 10, pady = 10)
        Label(self, text = 'Player Wins: ').grid(row = 4, column = 2, padx = 10, pady = 10)
        self.pcounts = Label(self, text = self._player_wins)
        self.pcounts.grid(row = 4, column = 3, padx = 10, pady = 10)
        self.pcounts.update_idletasks()
    
    def hitcb(self):
        """
        Appends card from top of deck to players hand, changes display of cards in players hand from blank to the new card. 
        """
    
        label = [self._pc3, self._pc4, self._pc5, self._pc6]
    
        if self._player_move < 4 and self.total(self._player_hand) <= 21:
            card = self._cards.deal(1)
            self._player_hand.extend(card)
            label[self._player_move].display('front', self._player_hand[self._player_move + 2]._id)
            self._player_move += 1
            
            if self.total(self._player_hand) > 21:
                self._hit.configure(state = DISABLED)
                showinfo(title = 'Game Over', message = 'You Lose, Total Over 21')
                self._player_move = 0
                self._dealer_wins += 1
    
    def standcb(self):
        """
        'flips' over dealers first card to front, compares total points of player and dealer to determine winner.
        """
        
        label = [self._dc3, self._dc4, self._dc5, self._dc6]
        
        while self.total(self._dealer_hand) < 17:
            card = self._cards.deal(1)
            self._dealer_hand.extend(card)
            label[self._dealer_move].display('front', self._dealer_hand[self._dealer_move + 2]._id)
            self._dealer_move += 1
         
        # prevent game being continued after stand command
        self._hit.configure(state = DISABLED)
        self._stand.configure(state = DISABLED)
        self._dc1.display('front', self._dealer_hand[0]._id)
        
        dealer_total, player_total = self.total(self._dealer_hand), self.total(self._player_hand)
        
        # winner determination
        if dealer_total <= 21 and player_total <= 21:
            if dealer_total > player_total:
                showinfo(title = 'Game Over', message = 'You Lose, Dealer Wins')
                self._dealer_wins += 1
            elif player_total > dealer_total:
                showinfo(title = 'Game Over', message = 'You Win!')
                self._player_wins += 1
            elif dealer_total > 21:
                showinfo(title = 'Game Over', message = 'You Win!')
                self._player_wins += 1
            else:
                showinfo(title = 'Game Over', message = "It's a tie")
        else:
            showinfo(title = 'Game Over', message = "It's a tie")
        
        
    def total(self, hand):
        """
        Calculates the total for a given hand.
        """
        points = 0
        n_aces = 0
        
        for card in hand:
            points += card.points()
            if card.rank() == 12:
                n_aces += 1
        
        while points > 21 and n_aces > 0: # if Ace is 1 or 11
            points -= 10
            n_aces -= 1
            
        return points

In [17]:
blackjack = Tk()
blackjack.geometry('510x400')
BlackjackFrame(blackjack).pack()

### <span style="color:teal">Extra Credit Ideas</span>

<span style="color:red; font-weight:bold;">Important:</span>  To earn extra credit points make sure you fill in the following markup cell to explain what you did so the graders will look for your extensions when they grade your project:

* Keeping count of number of games won and lost

Keep track of how many games the player has won or lost, display the counts in text labels

Let the user specify an initial amount of money they want to play with, and the amount they want to bet at the start of a game.  When the game is over add to or subtract from their balance.

Check to see if the player or dealer have "blackjack" (the hand has 21 points with only two cards).  A blackjack wins the game immediately.

In some casinos, the dealer or the player wins if they have 5 cards that total 21 or less, no matter what the other player’s total is (unless the other person has a blackjack).

#### Extra Credit: Monte Carlo 

The window in your Blackjack program has room for 6 cards for each player. Is this enough space? What is the probability of a player needing more than 6 cards? One way to figure this out is to use a technique called, appropriately enough, **Monte Carlo integration.**

Create a histogram with 10 bins, labeled 1 through 10, all initially set to 0. Then deal a series of hands, keeping track of how many cards you need to deal before the total is 22 or higher. For each hand update the bin that corresponds to the number of cards dealt; for example, if a hand reaches 25 on the 3rd card add one to the count in bin 3. If you repeat this process several million times you will start to get a pretty good idea of the shape of the probability distribution.

How many times did you get a hand that would have won a game with 5 cards, *i.e.* how many hands took 6 or more cards to go past 21? From this count can you calculate the probability of winning with 5 cards? Will your program ever see a hand that requires 7 or more images in a single row?
